# **PROYECTO FINAL**
**Daniela Navas**

## KNN

In [4]:
import random
import math
from collections import Counter

features = ["MAV","RMS","V","WL","SSC","IEMG","LOGVAR","PF","TP","SE","FR","BW","PSD","P2P"]
clases = ["reposo", "palma", "pinza", "puno"]

# -----------------------------------------
def leer_csv(nombre_archivo):
    with open(nombre_archivo, "r") as f:
        lineas = f.readlines()
    encabezados = lineas[0].strip().split(",")
    datos = []
    for linea in lineas[1:]:
        partes = linea.strip().split(",")
        if len(partes) != len(encabezados):
            continue
        registro = {encabezados[i]: partes[i] for i in range(len(encabezados))}
        datos.append(registro)
    return datos

# -----------------------------------------
def euclidean_distance(a, b):
    suma = 0.0
    for f in features:
        suma += (float(a[f]) - float(b[f])) ** 2
    return math.sqrt(suma)

# -----------------------------------------
def knn(train_set, instancia, k=3):
    distancias = []
    for elemento in train_set:
        dist = euclidean_distance(instancia, elemento)
        distancias.append((elemento, dist))
    distancias.sort(key=lambda x: x[1])
    vecinos = distancias[:k]
    etiquetas = [vecino[0]["MOV"] for vecino in vecinos]
    prediccion = Counter(etiquetas).most_common(1)[0][0]
    return prediccion

# -----------------------------------------
def dividir_dataset(dataset, porcentaje_entrenamiento=0.8, semilla=42):
    random.seed(semilla)
    copia = dataset[:]
    random.shuffle(copia)
    corte = int(len(copia) * porcentaje_entrenamiento)
    return copia[:corte], copia[corte:]

# -----------------------------------------
def matriz_confusion(y_verdadero, y_predicho):
    matriz = {real: {pred: 0 for pred in clases} for real in clases}
    for real, pred in zip(y_verdadero, y_predicho):
        matriz[real][pred] += 1
    return matriz

# -----------------------------------------
def metricas_clasificacion(matriz):
    print("\nMétricas por clase:")
    for clase in clases:
        TP = matriz[clase][clase]
        FP = sum(matriz[otra][clase] for otra in clases if otra != clase)
        FN = sum(matriz[clase][otra] for otra in clases if otra != clase)
        TN = sum(
            matriz[otra1][otra2]
            for otra1 in clases if otra1 != clase
            for otra2 in clases if otra2 != clase
        )
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        sensibilidad = TP / (TP + FN) if (TP + FN) > 0 else 0
        especificidad = TN / (TN + FP) if (TN + FP) > 0 else 0
        print(f"\nClase: {clase}")
        print(f"  Precisión     : {precision:.2f}")
        print(f"  Sensibilidad  : {sensibilidad:.2f}")
        print(f"  Especificidad : {especificidad:.2f}")

# -----------------------------------------
def ejecutar_knn(nombre_csv, k=3, seed=42):
    datos = leer_csv(nombre_csv)
    entrenamiento, prueba = dividir_dataset(datos, porcentaje_entrenamiento=0.8, semilla=seed)

    y_verdadero = []
    y_predicho = []

    for ejemplo in prueba:
        y_verdadero.append(ejemplo["MOV"])
        pred = knn(entrenamiento, ejemplo, k)
        y_predicho.append(pred)

    matriz = matriz_confusion(y_verdadero, y_predicho)

    print("\nMatriz de Confusión:")
    print("               Predicho")
    print("          ", "  ".join([f"{c:^7}" for c in clases]))
    print("Real")
    for real in clases:
        fila = " ".join([f"{matriz[real][pred]:^7}" for pred in clases])
        print(f"{real:<9} {fila}")

    metricas_clasificacion(matriz)


ejecutar_knn("dataset_completo.csv", k=3)



Matriz de Confusión:
               Predicho
           reposo    palma    pinza    puno  
Real
reposo      29       1       0       0   
palma        0      33       5       0   
pinza        2       3      19       1   
puno         0       3       3      21   

Métricas por clase:

Clase: reposo
  Precisión     : 0.94
  Sensibilidad  : 0.97
  Especificidad : 0.98

Clase: palma
  Precisión     : 0.82
  Sensibilidad  : 0.87
  Especificidad : 0.91

Clase: pinza
  Precisión     : 0.70
  Sensibilidad  : 0.76
  Especificidad : 0.92

Clase: puno
  Precisión     : 0.95
  Sensibilidad  : 0.78
  Especificidad : 0.99


---
## SVM

In [14]:
import random
import math

features = ["MAV","RMS","V","WL","SSC","IEMG","LOGVAR","PF","TP","SE","FR","BW","PSD","P2P"]
clases = ["reposo", "palma", "pinza", "puno"]

# -------------------------------------
def leer_csv(nombre_archivo):
    with open(nombre_archivo, "r") as f:
        lineas = f.readlines()
    encabezados = lineas[0].strip().split(",")
    datos = []
    for linea in lineas[1:]:
        partes = linea.strip().split(",")
        if len(partes) != len(encabezados):
            continue
        fila = {encabezados[i]: partes[i] for i in range(len(encabezados))}
        datos.append(fila)
    return datos

# -------------------------------------
def dividir_dataset(dataset, porcentaje_entrenamiento=0.8, semilla=42):
    random.seed(semilla)
    copia = dataset[:]
    random.shuffle(copia)
    corte = int(len(copia) * porcentaje_entrenamiento)
    return copia[:corte], copia[corte:]

# -------------------------------------
def convertir_a_vectores(dataset):
    X = []
    y = []
    for fila in dataset:
        x = [float(fila[f]) for f in features]
        X.append(x)
        y.append(fila["MOV"])
    return X, y

# -------------------------------------
# Entrena SVM para una clase contra el resto
def entrenar_svm_binario(X, y_binario, epochs=200, lr=0.005, lam=0.001):
    w = [0.0 for _ in range(len(X[0]))]
    b = 0.0
    for epoch in range(epochs):
        for i in range(len(X)):
            xi = X[i]
            yi = y_binario[i]
            margin = yi * (sum(w[j]*xi[j] for j in range(len(xi))) + b)
            if margin >= 1:
                for j in range(len(w)):
                    w[j] -= lr * (2 * lam * w[j])
            else:
                for j in range(len(w)):
                    w[j] -= lr * (2 * lam * w[j] - yi * xi[j])
                b += lr * yi
    return w, b

# -------------------------------------
# Multiclase: One-vs-Rest
def entrenar_svm_multiclase(X, y, clases):
    modelos = {}
    for clase in clases:
        y_bin = [1 if etiqueta == clase else -1 for etiqueta in y]
        w, b = entrenar_svm_binario(X, y_bin)
        modelos[clase] = (w, b)
    return modelos

# -------------------------------------
# Predicción SVM multiclase
def predecir_svm_multiclase(X, modelos):
    predicciones = []
    for xi in X:
        scores = {}
        for clase in modelos:
            w, b = modelos[clase]
            score = sum(w[j]*xi[j] for j in range(len(xi))) + b
            scores[clase] = score
        predicciones.append(max(scores, key=scores.get))
    return predicciones

# -------------------------------------
# Matriz de confusión y métricas
def matriz_confusion(y_real, y_pred):
    matriz = {real: {pred: 0 for pred in clases} for real in clases}
    for real, pred in zip(y_real, y_pred):
        matriz[real][pred] += 1
    return matriz

def metricas_clasificacion(matriz):
    print("\nMétricas por clase:")
    for clase in clases:
        TP = matriz[clase][clase]
        FP = sum(matriz[otra][clase] for otra in clases if otra != clase)
        FN = sum(matriz[clase][otra] for otra in clases if otra != clase)
        TN = sum(
            matriz[otra1][otra2]
            for otra1 in clases if otra1 != clase
            for otra2 in clases if otra2 != clase
        )
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        sensibilidad = TP / (TP + FN) if (TP + FN) > 0 else 0
        especificidad = TN / (TN + FP) if (TN + FP) > 0 else 0
        print(f"\nClase: {clase}")
        print(f"  Precisión     : {precision:.2f}")
        print(f"  Sensibilidad  : {sensibilidad:.2f}")
        print(f"  Especificidad : {especificidad:.2f}")

# -------------------------------------
def ejecutar_svm(nombre_csv, seed=42):
    datos = leer_csv(nombre_csv)
    train_data, test_data = dividir_dataset(datos, semilla=seed)

    X_train, y_train = convertir_a_vectores(train_data)
    X_test, y_test = convertir_a_vectores(test_data)

    modelos = entrenar_svm_multiclase(X_train, y_train, clases)
    predicciones = predecir_svm_multiclase(X_test, modelos)

    matriz = matriz_confusion(y_test, predicciones)

    print("\nMatriz de Confusión:")
    print("               Predicho")
    print("          ", "  ".join([f"{c:^7}" for c in clases]))
    print("Real")
    for real in clases:
        fila = " ".join([f"{matriz[real][pred]:^7}" for pred in clases])
        print(f"{real:<9} {fila}")

    metricas_clasificacion(matriz)

ejecutar_svm("dataset_completo.csv")



Matriz de Confusión:
               Predicho
           reposo    palma    pinza    puno  
Real
reposo      30       0       0       0   
palma        0      38       0       0   
pinza        8      13       0       4   
puno         1       4       0      22   

Métricas por clase:

Clase: reposo
  Precisión     : 0.77
  Sensibilidad  : 1.00
  Especificidad : 0.90

Clase: palma
  Precisión     : 0.69
  Sensibilidad  : 1.00
  Especificidad : 0.79

Clase: pinza
  Precisión     : 0.00
  Sensibilidad  : 0.00
  Especificidad : 1.00

Clase: puno
  Precisión     : 0.85
  Sensibilidad  : 0.81
  Especificidad : 0.96


---
## Naive Bayes

In [15]:
import random
import math

features = ["MAV","RMS","V","WL","SSC","IEMG","LOGVAR","PF","TP","SE","FR","BW","PSD","P2P"]
clases = ["reposo", "palma", "pinza", "puno"]

# -------------------------------------
def leer_csv(nombre_archivo):
    with open(nombre_archivo, "r") as f:
        lineas = f.readlines()
    encabezados = lineas[0].strip().split(",")
    datos = []
    for linea in lineas[1:]:
        partes = linea.strip().split(",")
        if len(partes) != len(encabezados):
            continue
        fila = {encabezados[i]: partes[i] for i in range(len(encabezados))}
        datos.append(fila)
    return datos

def dividir_dataset(dataset, porcentaje_entrenamiento=0.8, semilla=42):
    random.seed(semilla)
    copia = dataset[:]
    random.shuffle(copia)
    corte = int(len(copia) * porcentaje_entrenamiento)
    return copia[:corte], copia[corte:]

def convertir_a_vectores(dataset):
    X = []
    y = []
    for fila in dataset:
        x = [float(fila[f]) for f in features]
        X.append(x)
        y.append(fila["MOV"])
    return X, y

# -------------------------------------
# Naive Bayes
def calcular_parametros(X, y):
    params = {}
    for clase in clases:
        indices = [i for i in range(len(y)) if y[i] == clase]
        clase_features = [X[i] for i in indices]
        n = len(clase_features)
        medias = [sum(f[j] for f in clase_features) / n for j in range(len(features))]
        varianzas = []
        for j in range(len(features)):
            var = sum((f[j] - medias[j])**2 for f in clase_features) / n
            varianzas.append(var if var > 1e-6 else 1e-6)  # evitar división por cero
        params[clase] = {"media": medias, "var": varianzas, "prior": n / len(y)}
    return params

def gaussiana(x, media, var):
    exp = math.exp(-(x - media)**2 / (2 * var))
    return (1 / math.sqrt(2 * math.pi * var)) * exp

def predecir(X, params):
    predicciones = []
    for xi in X:
        probs = {}
        for clase in clases:
            media = params[clase]["media"]
            var = params[clase]["var"]
            prior = params[clase]["prior"]
            prob = math.log(prior)
            for j in range(len(xi)):
                p = gaussiana(xi[j], media[j], var[j])
                prob += math.log(p if p > 1e-10 else 1e-10)  # evitar log(0)
            probs[clase] = prob
        predicciones.append(max(probs, key=probs.get))
    return predicciones

# -------------------------------------
# Métricas
def matriz_confusion(y_real, y_pred):
    matriz = {real: {pred: 0 for pred in clases} for real in clases}
    for real, pred in zip(y_real, y_pred):
        matriz[real][pred] += 1
    return matriz

def metricas_clasificacion(matriz):
    print("\nMétricas por clase:")
    for clase in clases:
        TP = matriz[clase][clase]
        FP = sum(matriz[otra][clase] for otra in clases if otra != clase)
        FN = sum(matriz[clase][otra] for otra in clases if otra != clase)
        TN = sum(
            matriz[otra1][otra2]
            for otra1 in clases if otra1 != clase
            for otra2 in clases if otra2 != clase
        )
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        sensibilidad = TP / (TP + FN) if (TP + FN) > 0 else 0
        especificidad = TN / (TN + FP) if (TN + FP) > 0 else 0
        print(f"\nClase: {clase}")
        print(f"  Precisión     : {precision:.2f}")
        print(f"  Sensibilidad  : {sensibilidad:.2f}")
        print(f"  Especificidad : {especificidad:.2f}")

# -------------------------------------
def ejecutar_naive_bayes(nombre_csv, seed=42):
    datos = leer_csv(nombre_csv)
    train_data, test_data = dividir_dataset(datos, semilla=seed)

    X_train, y_train = convertir_a_vectores(train_data)
    X_test, y_test = convertir_a_vectores(test_data)

    params = calcular_parametros(X_train, y_train)
    predicciones = predecir(X_test, params)

    matriz = matriz_confusion(y_test, predicciones)

    print("\nMatriz de Confusión:")
    print("               Predicho")
    print("          ", "  ".join([f"{c:^7}" for c in clases]))
    print("Real")
    for real in clases:
        fila = " ".join([f"{matriz[real][pred]:^7}" for pred in clases])
        print(f"{real:<9} {fila}")

    metricas_clasificacion(matriz)

ejecutar_naive_bayes("dataset_completo.csv", seed=42)



Matriz de Confusión:
               Predicho
           reposo    palma    pinza    puno  
Real
reposo      30       0       0       0   
palma        0      34       4       0   
pinza        1       3      21       0   
puno         0       3       2      22   

Métricas por clase:

Clase: reposo
  Precisión     : 0.97
  Sensibilidad  : 1.00
  Especificidad : 0.99

Clase: palma
  Precisión     : 0.85
  Sensibilidad  : 0.89
  Especificidad : 0.93

Clase: pinza
  Precisión     : 0.78
  Sensibilidad  : 0.84
  Especificidad : 0.94

Clase: puno
  Precisión     : 1.00
  Sensibilidad  : 0.81
  Especificidad : 1.00


---
## Random Forest